## Customer Data

There's a few filetypes to look at
* csv
* text (tab deliminated)
* zip

1. First, we'll READ in each data source
2. Next, we'll do some EDA to look at the shape of each datasource
3. Finally, we'll determine which meta data fields that'll need to be renamed, NULL values, and possible conversions required.

## Summary for Conversions
* [x] All data frames have the same column names
* [ ] Country will need to be formatted
* [ ] City has a '#' in a few city names to be removed
* [ ] State will need Fuzzy Matching to fix spelling errors

### READ: Multiple csv inputs (Customer Data West)

In [46]:
# Read in the multi csv files from the CustomerData_West folder and combine them into one dataframe
import pandas as pd
pd.set_option('display.max_rows', None)
try:
    path = 'C:/Users/User/Documents/pdi-mc-examples/pdi-pentaho/sample-data/customers/CustomerData_West/'
    customer_data_west = pd.concat(map(pd.read_csv, 
                       [path + 'CustomerData_West_Arizono.csv',
                       path + 'CustomerData_West_California.csv',
                       path + 'CustomerData_West_Colorado.csv',
                       path + 'CustomerData_West_Others.csv',
                       path + 'CustomerData_West_Washington.csv']))

    #info shows us the count of entires, column names, and non-null counts
    customer_data_west.info()

    #dtypes shows more info about each column and their data type (object is string and int64 is an integer)
    customer_data_west.dtypes
except Exception as e:
    print(e)

<class 'pandas.core.frame.DataFrame'>
Index: 254 entries, 0 to 37
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Customer ID    254 non-null    object
 1   Customer Name  254 non-null    object
 2   Segment        254 non-null    object
 3   Age            254 non-null    int64 
 4   Country        254 non-null    object
 5   City           254 non-null    object
 6   State          254 non-null    object
 7   Postal Code    254 non-null    int64 
 8   Region         254 non-null    object
dtypes: int64(2), object(7)
memory usage: 19.8+ KB


In [47]:
# Find NULL values
customer_data_west.isnull().sum()

Customer ID      0
Customer Name    0
Segment          0
Age              0
Country          0
City             0
State            0
Postal Code      0
Region           0
dtype: int64

### READ: Excel Data (Customer Data East)

In [48]:
try:
    path = 'C:/Users/User/Documents/pdi-mc-examples/pdi-pentaho/sample-data/customers/'
    customer_data_east = pd.concat(map(pd.read_excel, 
                       [path + 'CustomerData_East.xlsx']))

    customer_data_east.info()
    customer_data_east.dtypes
except Exception as e:
    print(e)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Customer ID    220 non-null    object
 1   Customer Name  220 non-null    object
 2   Segment        220 non-null    object
 3   Age            220 non-null    object
 4   Country        220 non-null    object
 5   City           220 non-null    object
 6   State          220 non-null    object
 7   Postal Code    220 non-null    int64 
 8   Region         220 non-null    object
dtypes: int64(1), object(8)
memory usage: 15.6+ KB


### READ: Tab Seperated Data (Customer Data Central)

In [49]:
try:
    path = 'C:/Users/User/Documents/pdi-mc-examples/pdi-pentaho/sample-data/customers/'
    customer_data_central = pd.concat(map(pd.read_table, 
                       [path + 'CustomerData_Central.txt']))

    customer_data_central.info()
    customer_data_central.dtypes
except Exception as e:
    print(e)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Customer ID    184 non-null    object
 1   Customer Name  184 non-null    object
 2   Segment        184 non-null    object
 3   Age            184 non-null    object
 4   Country        184 non-null    object
 5   City           184 non-null    object
 6   State          184 non-null    object
 7   Postal Code    184 non-null    int64 
 8   Region         184 non-null    object
dtypes: int64(1), object(8)
memory usage: 13.1+ KB


### READ: Zip Data (Customer Data South)

In [50]:
try:
    path = 'C:/Users/User/Documents/pdi-mc-examples/pdi-pentaho/sample-data/customers/'
    customer_data_south = pd.concat(map(pd.read_csv, 
                       [path + 'CustomerData_South.zip']))

    customer_data_south.info()
    customer_data_south.dtypes
except Exception as e:
    print(e)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Customer ID    133 non-null    object
 1   Customer Name  133 non-null    object
 2   Segment        133 non-null    object
 3   Age            133 non-null    object
 4   Country        133 non-null    object
 5   City           133 non-null    object
 6   State          133 non-null    object
 7   Postal Code    133 non-null    int64 
 8   Region         133 non-null    object
dtypes: int64(1), object(8)
memory usage: 9.5+ KB


## Exploratory Data Analysis (EDA)

### Check the Meta Data of the Dataframes

In [51]:
# Compare column names of the data frames
data_frames = {
    'Customer Data West': customer_data_west,
    'Customer Data East': customer_data_east,
    'Customer Data Central': customer_data_central,
    'Customer Data South': customer_data_south
}

column_names = [df.columns for df in data_frames.values()]
all_columns_same = all(names.equals(column_names[0]) for names in column_names)

if all_columns_same:
    print("All data frames have the same column names.")
else:
    print("Not all data frames have the same column names.")
    for i, names in enumerate(column_names):
        if not names.equals(column_names[0]):
            mismatched_columns = names[~names.isin(column_names[0])]
            print(f"Columns in {list(data_frames.keys())[i]} that do not match:")
            print(mismatched_columns)
            print()


All data frames have the same column names.


### Check each of the Dataframes for possible data conversions

In [52]:
country_unique_values = customer_data_west['Country'].value_counts().sort_index()
state_unique_values = customer_data_west['State'].value_counts().sort_index()

print("Unique values and their counts for 'Country' column in alphabetical order:")
print(country_unique_values)

print("\nUnique values and their counts for 'State' column in alphabetical order:")
print(state_unique_values)


Unique values and their counts for 'Country' column in alphabetical order:
Country
US                           21
USA                          38
United States               175
United States of America     20
Name: count, dtype: int64

Unique values and their counts for 'State' column in alphabetical order:
State
Arizona        21
Cakifornia      1
California    157
Californis      1
Caligornia      1
Colorado       20
Nevada          2
New Mexico      4
Oregon          3
Utah            6
Washington     38
Name: count, dtype: int64


In [53]:
# function get unique values for a column in a dataframe and sort them in alphabetical order
def get_unique_values(df, column):
    unique_values = df[column].value_counts().sort_index()
    return unique_values

# get_unique_values(customer_data_west, 'Country')

In [54]:
# Quick loop calling the function to see what might be duplicate values
# for column in customer_data_west.columns:
    # unique_values = get_unique_values(customer_data_west, column)
    # print(f"Unique values and their counts for '{column}')")
    # print(unique_values)
    # print()

In [55]:
West_Country = get_unique_values(customer_data_west, 'Country')
West_State = get_unique_values(customer_data_west, 'State')
West_City = get_unique_values(customer_data_west, 'City')

print(West_Country)
print(West_State)
print(West_City)

Country
US                           21
USA                          38
United States               175
United States of America     20
Name: count, dtype: int64
State
Arizona        21
Cakifornia      1
California    157
Californis      1
Caligornia      1
Colorado       20
Nevada          2
New Mexico      4
Oregon          3
Utah            6
Washington     38
Name: count, dtype: int64
City
#Apple Valley        1
#Salinas             2
#Whittier            1
Albuquerque          2
Arvada               1
Aurora               6
Brentwood            1
Broomfield           1
Carlsbad             1
Colorado Springs     1
Concord              1
Costa Mesa           1
Denver               3
Des Moines           1
Encinitas            1
Escondido            1
Fairfield            1
Fort Collins         1
Fresno               3
Gilbert              2
Glendale             2
Hesperia             1
Huntington Beach     1
Inglewood            1
Kent                 1
La Quinta            1
Lagun